In [2]:
import os
import pandas as pd
from utils.model_analysis_utils import save_embeddings,summarize_scan_embs,create_res_df_model
from data.MEG import get_data
from models.base_models import  LPModel

In [95]:
import torch
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [4]:
study_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\pats_CogTr1\L3\HGCN_full_findc_plvid_dp\e100_p6_lr0.021_val_excl_group_strchinp95_strchloss95_b8'
study_dir=r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\study\meg\lp\all_pats\L3\HGCN_full_0.54c_id_dp\e100_p6_lr0.021_vpct0.3_tpct0.2_stretch98_b8'

In [6]:
root = r'C:\Users\coleb\OneDrive\Desktop\Fall 2021\Neuro\hyperBrain\outputs\link_preds'

In [79]:
dimensions = os.listdir(root)
# dimensions = [dimensions[1]]
dimensions

['L2', 'L3', 'L4', 'L6', 'L8']

In [76]:

geom_type_dict= {'Poincare (Shallow)':'alpha_full_poinc','GCN':'alpha_full_euc','HGCN(-1)':'alpha_full_setc','HGCN(flex)':'alpha_full_findc'}
stat_cols = ['mAPs','Mean Ranks','ROC']
key_stat_col = 'mAPs'

In [91]:
results_df_list = []
for d_name in dimensions:
    assert len(d_name)==2
    d_folder = os.path.join(root,d_name)
    l = d_name[1]
    assert int(l)<10
    print(l)
    assert os.path.exists(d_folder),d_folder
    
    for g_type, g_name in geom_type_dict.items():
        g_name_dim = g_name+f'_{l}d'
        g_folder = os.path.join(d_folder,g_name_dim)
        if not os.path.exists(g_folder):
            continue
#         assert os.path.exists(g_folder),g_folder
#         print(g_folder)
        result_df_path = os.path.join(g_folder,[f for f in os.listdir(g_folder) if 'meg_lp_alpha' == f[:len('meg_lp_alpha')]][-1])
#         print(result_df_path)
        result_df = pd.read_csv(result_df_path)
        result_df_stats = result_df[stat_cols].mean().to_frame().transpose()
#         print(result_df_path)
#         print(result_df_stats)
        if result_df_stats.isnull().values.any():
            print(result_df)
            assert False
            
            
        if g_type == 'HGCN(flex)':
            running_c=0
            total=0
            for f in os.listdir(g_folder):
                model_root = os.path.join(g_folder,f)
                if not os.path.isdir(model_root):
                    continue
                model_path = os.path.join(model_root,"{}.pt".format('model'))  ### args should be saved with the model
                # printmodel_path,'model path'
                model = torch.load(model_path,map_location=torch.device('cpu'))
                r=model.dc.r.item()
                t=model.dc.t.item()
                c=model.c.item()
                dims=model.args.output_dim
#                 for lay in range(model.args.num_layers):
#                     print(lay,model.encoder.curvatures[lay].detach().numpy()[0],'layer of c',l)
# #                     print(c,'listed c')
#                 print(lay,model.encoder.curvatures[model.args.num_layers-1][0].detach().numpy())
                running_c+=model.encoder.curvatures[model.args.num_layers-1].detach().numpy()
                total+=1
            found_c = running_c/total
#             print(found_c)
#             sk
        elif g_type == 'HGCN(-1)':
            found_c = 1
            
        elif g_type == 'GCN':
            found_c = 0
        elif g_type == 'Poincare (Shallow)':
            found_c = 1
        else:
            raise Exception(f'Not Found: {g_type}')
#         print(found_c)
        result_df_stats['Geometry']=g_type
        result_df_stats['dim']=int(l)
        result_df_stats['-K']=found_c
        results_df_list.append(result_df_stats)
    
results_df_full = pd.concat(results_df_list)
results_df_full

2
3
4
6
8


,mAPs,Mean Ranks,ROC,Geometry,dim,-K
0,0.725021,21.626020,0.814463,Poincare (Shallow),2,1.000000
0,0.563116,21.707912,0.847962,GCN,2,0.000000
0,0.799440,17.146872,0.940828,HGCN(-1),2,1.000000
0,0.818765,14.954579,0.960792,HGCN(flex),2,0.982341
0,0.748285,20.820741,0.821682,Poincare (Shallow),3,1.000000
0,0.701568,17.868042,0.903556,GCN,3,0.000000
0,0.939286,13.733084,0.983232,HGCN(-1),3,1.000000
0,0.951668,13.332801,0.987464,HGCN(flex),3,0.582297
0,0.750611,20.756798,0.822179,Poincare (Shallow),4,1.000000
0,0.793427,15.253697,0.940682,GCN,4,0.000000


In [117]:
# pivot table
unique_dims = [2,3,4,6,8]
unique_geoms = geom_type_dict.keys()
key_stat_col = 'mAPs'
rows = []
for g in unique_geoms:
    sub_df = results_df_full[results_df_full['Geometry']==g]
    g_vals = [g]
    cols = ['Geometry']+[str(d) for d in unique_dims]
    for d in unique_dims:
        row = sub_df[sub_df['dim']==d]
        print(row.shape)
        assert row.shape[0]==1,(g,d)
        row= row.iloc[0]
        val = str(round(row[key_stat_col],3))
        if g =='HGCN(flex)':
            val+=f" (K=-{round(row['-K'],2)})"
        print(val)
        g_vals.append(val)
    row = pd.DataFrame(columns=cols,data=np.array([g_vals]))
    rows.append(row)
pivot = pd.concat(rows)
pivot

(1, 6)
0.725
(1, 6)
0.748
(1, 6)
0.751
(1, 6)
0.751
(1, 6)
0.751
(1, 6)
0.563
(1, 6)
0.702
(1, 6)
0.793
(1, 6)
0.813
(1, 6)
0.827
(1, 6)
0.799
(1, 6)
0.939
(1, 6)
0.949
(1, 6)
0.951
(1, 6)
0.951
(1, 6)
0.819 (K=-0.98)
(1, 6)
0.952 (K=-0.58)
(1, 6)
0.955 (K=-0.5)
(1, 6)
0.96 (K=-0.44)
(1, 6)
0.964 (K=-0.53)


,Geometry,2,3,4,6,8
0,Poincare (Shallow),0.725,0.748,0.751,0.751,0.751
0,GCN,0.563,0.702,0.793,0.813,0.827
0,HGCN(-1),0.799,0.939,0.949,0.951,0.951
0,HGCN(flex),0.819 (K=-0.98),0.952 (K=-0.58),0.955 (K=-0.5),0.96 (K=-0.44),0.964 (K=-0.53)
